# House Keeping

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf # type: ignore
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import chi2_contingency # type: ignore
import statsmodels.formula.api as smf # type: ignore

# Reading

Textbook Page 127-139.

# Correlation and Causation

- Some examples of correlation and causation.
  - Salary and Education
  - Asset Return and Volatility
  - Economic growth and Oil price change
  - Unemployment and Inflation
  - Sales and Promotion
  - Investment and Consumption
  - Hotel price and Sales


- Econometrics are more into causality, while ML loves prediction.

- Only when causality is established we can make **counterfactual prediction**.

## Randomised Controlled Trials (RCT)

The terminology is easy to understand. Control for everything else and only change something you want to understand to see the difference. It is also labelled as **AB testing** in the textbook and business optimisation in general.

- **Treatment Effect (TE)**
- **Treatment Variable**: $d$, which can be discrete or continuous.
  - enter a program or not
  - price cut for sale
  - new drug intake amount

AB trial is also known as *completely randomized design*.
- A is control group and B is treatment group
- Average Treatment Effect（ATE）
  $$ATE = E(y\mid d=1)-E(y\mid d=0)$$
- $d$ must be independent of all other factors.
- Estimate
  $$\widehat{ATE}=\overline{y}_1-\overline{y}_0$$
- The standard error is the same as the SE of comparing two independent sample means in QM1. See (5.3) from the textbook. We briefly mention it here for completeness.

# [The Oregon Health Insurance Experiment](https://www.nber.org/programs-projects/projects-and-centers/oregon-health-insurance-experiment?page=1&perPage=50)

2008, Oregon expand its coverage for Medicaid. Because demand is higher than supply, the state used lottery to allocate. This is the Oregon Health Insurance Experiment (OHIE), a randomised controlled AB trial to meausre the treatment effect of Medicaid eligibility (note: NOT access).

- people were tracked for 12 months.


We will work with a data file from an actual scientific publication. They shared key data in various files in the Stata statistical program's main data format. So we'll load that.

- `person_id`  is a key for the people in the study.
- `treatment` is in the description file. This is the $d$ variable!

- For more details, you may visit their [NBER page](https://www.nber.org/research/data/oregon-health-insurance-experiment-data)

We are interested in the effect of health insurance on increased costs and utilization (health would be longer term).

For details, see
- [description codebook](https://www.dropbox.com/s/l8oymjphxb5sojl/oregonhie_descriptivevars_codebook.pdf?dl=0)
- [program codebook](https://www.dropbox.com/s/03f5xyf1top6a2z/oregonhie_stateprograms_codebook.pdf?dl=0)
- [survey 12 month codebook](https://www.dropbox.com/s/ut3uzm1e431a5v5/oregonhie_survey12m_codebook.pdf?dl=0)

In [2]:
descr = pd.read_csv("https://www.dropbox.com/scl/fi/0fdjb40d8p2opcvkdrhp9/descr.csv?rlkey=9mlcyhwwt0d4lbtovb023qjp2&dl=1")
prgm = pd.read_csv("https://www.dropbox.com/scl/fi/29yal1nt9we4kbdvsx4bc/prgm.csv?rlkey=l2dkn0hsgq7401a8fbgicri9n&dl=1")
s12 = pd.read_csv("https://www.dropbox.com/scl/fi/6t7u5d3a9bxnnev2xthog/s12.csv?rlkey=foauwm8b5w4mahhfcfqp6narb&dl=1")

## description file

In [3]:
descr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74922 entries, 0 to 74921
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   person_id          74922 non-null  int64 
 1   household_id       74922 non-null  int64 
 2   treatment          74922 non-null  object
 3   draw_treat         29834 non-null  object
 4   draw_lottery       74922 non-null  object
 5   applied_app        29799 non-null  object
 6   approved_app       29799 non-null  object
 7   dt_notify_lottery  74922 non-null  object
 8   dt_retro_coverage  74922 non-null  object
 9   dt_app_decision    18101 non-null  object
 10  postn_death        74922 non-null  object
 11  numhh_list         74922 non-null  object
 12  birthyear_list     74922 non-null  int64 
 13  have_phone_list    74922 non-null  object
 14  english_list       74922 non-null  object
 15  female_list        74921 non-null  object
 16  first_day_list     74922 non-null  objec

|variable name | label |
|--------------|--------|
| person_id | personal identifier | 
| household_id | household identifier|
| treatment    | **selected** in the lottery|
| draw_treat  | Lottery draw for thoese selected|
|  draw_lottery |     Lottery draw fro thoese selected, "matched" draw. This is the  *control group*|
|   applied_app | Submitted an application to OHP|
|   approved_app | Applicatin approved |
|   dt_notify_lottery | Lottery draw date |
|  dt_retro_coverage  | insurance begin date, retroactively after approval|
|  dt_app_decision    | data for application decision |
|  postn_death        | death post notificaiton date |
| numhh_list          | number of people in hoursehold|
|  birthyear_list     | birth year |
| have_phone_list    | gave a phone number? |
|  english_list       | request English material? |
|  female_list        | gender |
| first_day_list     | signed up for lottery list on the first day. *important psychological date*|
| last_day_list      | signed up for lottery list on the last day. *important psychological date*|
| pobox_list      | gave a PO box? |
|  self_list          | self signed up for lottery list?|
| week_list         | week of lottery sign up. *patience?*|
| zip_msa_list | Zip code in metropolitan statistical area?|

In [4]:
descr.head().T

,0,1,2,3,4
person_id,1,2,3,4,5
household_id,100001,100002,100003,100004,100005
treatment,Selected,Selected,Not selected,Not selected,Selected
draw_treat,Draw 7: selected in lottery 08/01/2008,Draw 6: selected in lottery 07/01/2008,NaN,NaN,Draw 7: selected in lottery 08/01/2008
draw_lottery,Lottery Draw 7,Lottery Draw 6,Lottery Draw 2,Lottery Draw 8,Lottery Draw 7
applied_app,Submitted an Application to OHP,Did NOT submit an application to OHP,NaN,NaN,Did NOT submit an application to OHP
approved_app,No,No,NaN,NaN,No
dt_notify_lottery,2008-08-12,2008-07-14,2008-04-07,2008-09-11,2008-08-12
dt_retro_coverage,2008-09-08,2008-08-08,2008-04-08,2008-10-08,2008-09-08
dt_app_decision,2008-12-31,NaN,NaN,NaN,NaN


In [6]:
descr.zip_msa_list.unique()

array(['Zip code of residence in a MSA',
       'Zip code of residence NOT in a MSA', nan], dtype=object)

## program file

In [5]:
prgm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74922 entries, 0 to 74921
Data columns (total 36 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   person_id                      74922 non-null  int64  
 1   ohp_all_ever_matchn_30sep2009  74922 non-null  object 
 2   ohp_all_ever_firstn_survey0m   58405 non-null  object 
 3   ohp_all_ever_firstn_survey6m   13024 non-null  object 
 4   ohp_all_ever_inperson          20745 non-null  object 
 5   ohp_all_ever_firstn_30sep2009  74922 non-null  object 
 6   ohp_all_end_30sep2009          74922 non-null  object 
 7   ohp_all_end_survey0m           58405 non-null  object 
 8   ohp_all_end_survey6m           13024 non-null  object 
 9   ohp_all_end_inperson           20745 non-null  object 
 10  ohp_all_at_12m                 58405 non-null  object 
 11  ohp_all_mo_matchn_30sep2009    74922 non-null  object 
 12  ohp_all_mo_firstn_survey0m     58405 non-null 

｜ variable | label |
|---|---|
|  person_id | individual identifier|
|   ohp_all_ever_matchn_30sep2009  | ever enrolled in Medicaid? from notification data to 30Sep2009|
|  ohp_all_ever_firstn_survey0m   | ever enrolled in Medicaid? from 10Mar2008 to 30Aug2008. *dates are meaningful, but not individual dependent*|
|  ohp_all_ever_firstn_survey6m | ever enrolled in Medicaid? from 10Mar2008 to 28Feb2009. |
| ohp_all_ever_inperson | ever enrolled in Medicaid? from 10Mar2008 until in-person interview. For in-person interview only|
|  ohp_all_ever_firstn_30sep2009   | ever enrolled in Medicaid? from 10Mar2008 to 30Sep2009. |
|  ohp_all_end_30sep2009 | Enrolled? on 30Sep2009 |
| ohp_all_end_survey0m. | Enrolled? on 30Aug2008 |
|   ohp_all_end_survey6m |           Enroled? on 28Feb2009 |
|   ohp_all_end_inperson | Enrolled? on in-person interview |
|  ohp_all_at_12m | enrolled? when they return 12m mail survey |
|  ohp_all_mo_matchn_30sep2009 | number of months on Medicaid between matched notificaiton date and 30Sep2009|
| ohp_all_mo_firstn_survey0m   | number of months on Medicaid between 10Mar2008 to 30Aug2008 |
|  ohp_all_mo_firstn_survey6m   | number of months on Medicaid between 10Mar2008 to 28Feb2009|
|  ohp_all_mo_inperson           |  number of months on Medicaid between 10Mar2008 and in-person interview
|  ohp_all_mo_firstn_30sep2009  |   number of months on Medicaid between from 10Mar2008 to 30Sep2009 |
|  ohp_all_mo_12m             |    number of months on Medicaid during 6 months prior to 12m survey return |
| ohp_std_ever_matchn_30sep2009 | Enroled in OHP standard (lottery)? from notification data to 30Sep2009 |
|  ohp_std_ever_inperson       |  Enroled in OHP standard (lottery)? from 10Mar2008 until in-person interview
| ohp_std_ever_firstn_30sep2009 |  Enroled in OHP standard (lottery)? from 10Mar2008 to 30Sep2009|
| - |  Supplemental Nutrition Assistance Program (SNAP) provides monthly food benefits to help you buy healthy food|
| snap_ever_prenotify07      |   In SNAP between 01Jan2007 and notificaiton date | 
| snap_ever_presurvey12m      |   In SNAP between 01Jan2007 and 10Mar2008 if in 12m sample | 
|  snap_ever_matchn_30sep2009  |   In SNAP between notification data to 30Sep2009 |
|  snap_ever_firstn_survey12m  |   In SNAP between 10Mar2008 to 30Sep2009 |
| snap_tot_hh_prenotify07      |  total household benefit from SNAP between 01Jan2007 and notificaiton date |
| snap_tot_hh_presurvey12m     |  total household benefit from SNAP between 01Jan2007 and 10Mar2008 if in 12m sample | 
| snap_tot_hh_30sep2009        |  total household benefit from SNAP between notification data to 30Sep2009 |
|  snap_tot_hh_firstn_survey12m  |  total household benefit from SNAP between 10Mar2008 to 30Sep2009 |
| - | TANF provides cash benefits to low-income families and new parents|
| tanf_ever_prenotify07        | Ever on TANF? between 01Jan2007 and notificaiton date | 
|  tanf_ever_presurvey12m       | Ever on TANF? between 01Jan2007 and 10Mar2008 if in 12m sample | 
| tanf_ever_matchn_30sep2009     | Ever on TANF? between notification data to 30Sep2009 |
|  tanf_ever_firstn_survey12m    | Ever on TANF? between 10Mar2008 to 30Sep2009 |
| tanf_tot_hh_prenotify07         | Total household benefit from TANF? between 01Jan2007 and notificaiton date |
|  tanf_tot_hh_presurvey12m     | Total household benefit from TANF?  between 01Jan2007 and 10Mar2008 if in 12m sample | 
|  tanf_tot_hh_30sep2009        | Total household benefit from TANF? between notification data to 30Sep2009 |
|  tanf_tot_hh_firstn_survey12m  | Total household benefit from TANF? between 10Mar2008 to 30Sep2009|

In [4]:
prgm.head().T

,0,1,2,3,4
person_id,1,2,3,4,5
ohp_all_ever_matchn_30sep2009,NOT enrolled,Enrolled,NOT enrolled,Enrolled,NOT enrolled
ohp_all_ever_firstn_survey0m,NOT enrolled,Enrolled,NOT enrolled,NaN,NOT enrolled
ohp_all_ever_firstn_survey6m,NaN,NaN,NaN,NaN,NOT enrolled
ohp_all_ever_inperson,NaN,NaN,NaN,NaN,NOT enrolled
ohp_all_ever_firstn_30sep2009,NOT enrolled,Enrolled,NOT enrolled,Enrolled,NOT enrolled
ohp_all_end_30sep2009,NOT enrolled,NOT enrolled,NOT enrolled,Enrolled,NOT enrolled
ohp_all_end_survey0m,NOT enrolled,Enrolled,NOT enrolled,NaN,NOT enrolled
ohp_all_end_survey6m,NaN,NaN,NaN,NaN,NOT enrolled
ohp_all_end_inperson,NaN,NaN,NaN,NaN,NOT enrolled


## survey 12m file

In [3]:
s12.info()
# too many to show

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74922 entries, 0 to 74921
Columns: 150 entries, person_id to needmet_dent_cor_12m
dtypes: float64(21), int64(1), object(128)
memory usage: 85.7+ MB


In [4]:
s12.columns

Index(['person_id', 'sample_12m', 'dt_mail_12m', 'dt_returned_12m',
       'ret_mode_12m', 'returned_12m', 'surv_lang_12m', 'in_survey_12m',
       'sample_12m_resp', 'sample_12m_nonresp',
       ...
       'live_alone_12m', 'live_parents_12m', 'live_relatives_12m',
       'live_friends_12m', 'live_other_12m', 'hhsize_12m', 'num19_12m',
       'needmet_med_cor_12m', 'needmet_rx_cor_12m', 'needmet_dent_cor_12m'],
      dtype='object', length=150)

| variable | label|
|---|---|
| person_id | identifier| 
| sample_12m | 12 month survey? |
| dt_mail_12m | date of first survey mailing 12m |
| dr_returned_12m | survey return date|
| ret_mode_12m | return method |
|returned_12m | returned survey |
|surv_lang_12m | Survey language |
|in_survey_12m | has 12m survey data|
|sample_12m_resp | responder with nonzero weight? | 
| sample_12m_nonresp | non-responder with nonzero weight?|
| sample_12m_nonzeroweight | nonzero weight invididual |
| weight_intensive_12m | weight for some non-responders. **The RM data has weight too**|
| *other weights| weight_newlottery_12m, weight_12m. **weights** are used to find the *unbiased* target| 
| wave_survey12m | multiple waves of survey |
| protocal_12m | intensive follow up indicator |
| Q1 health insurance cover | ins_any_12m， ins_ohp_12m, ins_medicare_12m, ins_employer_12m, ins_privpay_12m, ins_othcov_12m, ins_noins_12m, ins_private_12m, ins_other_12m. Some variables are derived|
| Q2 isn_month_12m| how many of last 6 months did you have insurance?|
|Q3 usual_place_12m | any usual place for medical care? |
|Q4 usual_care_12m | specific place for Q3. usual_clinic_12m is derived from it. |
|Q5 usual_doc_12m | Do you have one person as personal doctor?|
|Q6 need_med_12m | Did you need medical care in the last 6 month?|
| Q7 needmet_qn_med_12m | if you needed medicare care, did you get it?. Q6+Q7 derive *needmet_med_12m*|
|Q8 reasons for without needed medical care | reason_care_cost_12m, reason_care_ins_12m, reason_care_doc_12m, reason_care_owe_12m, reason_care_apt_12m, reason_care_closed_12m, reason_care_nodoc_12m, reason_Care_other_12m, reason_care_dont_12m |
| Q9 need_rx_12m | need presciption medications in the last 6 months?|
|Q10 needmet_qn_rx_12m | based on Q9, if needed, did you get it? Q9+Q10 derive *neemet_rx_12m*|
|Q11 reason for without medication (most recent) | reason_rx_cost_12m, reason_rx_ins_12m, reason_rx_doc_12m, reason_rx_get_12m, reaason_rx_pharm_12m, reason_rx_other_12m, reason_rx_dont_12m|
|rx_any_12m| =1 if Q12>0 (taking any prescription medication?)|
|Q12 rx_num_mod_12m | number of prescription meds currently taking |
|Q13 need_dent_12m| needed dental care in the last 6 months?|
|Q14 needmet_qn_dent_12m| if needed dental care, did you get it? Q13+Q14 derive *needmet_dent_12m*|
|doc_any_12m | =1 if Q15>0. See a doctor?|
|Q15 doc_num_mod_12m | number of care visits in the last 6 month|
|er_any_12m | =1 if Q16>0. visit ER?|
|Q16 er_num_mod_12m |number of ER vistis |
|Q17 reason to ER | reason_er_need_12m, reason_er_insure_12m, reason_er_closed_12m (clear typo in the file), reason_er_apt_12m, reason_er_doc_12m, reason_er_copay_12m, reason_er_go_12m, reason_er_other_12m, reason_er_rx_12m, reason_er_dont_12m. They derive *er_noner_12m*|
|hosp_any_12m | =1 if Q18>0 |
|Q18 hosp_num_mod_12m |  number of times visit a hopital as a patient at leaset overnight|
|Q19 med_qual_12m| quality of medical care recieved in the past 6 months |
| Q20 cost_any_oop_12m | paid out of pocket costs for medical care in the past 6 months?|
|Q21 OOP payment| cost_doc_oop_12m, cost_er_oop_12m, cost_rx_oop_12m,
cost_oth_oop_12m. They derive *cost_tot_oop_12m*|
|Q22 cost_any_owe_12m| owe money b/c medical expenses? *cost_tot_owe_12m* is the amount|
|Q23 cost_borrow_12m| borrow money to pay health care bill in the last 6 months?|
|Q24 cost_refused_12m| have you been refused care because you owed money for a past treatment?|
|Q25 happiness_12m| overall happiness|
|Q26 health_gen_12m| self-evaluated health. It derives *health_gen_bin_12m*|
|Q27 health_chg_12m| how has your health changed in the past 6 months. It derives *health_chg_bin_12m*|
|Q28 baddays_phys_12m| number of days (past 30 days) in bad physical health|
|Q29 baddays_ment_12m| number of days (past 30 days) in bad mental health|
|Q30 baddays_tot_12m| umber of days (past 30 days) in bad health|
|Q31 health_work_12m| health affect your work?|
|Q32 physical_act_12m| are you phisically more, same or less active than your peer?|
|Q33 dep_interst_12m| how often have you been disinterested in doing things in the past 2 weeks?|
|Q34 dep_sad_12m| How often have you felt down, depressed, hopeless in the past 2 weeks |
|Q35 informed disease| dia_dx_12m, ast_dx_12m, hbp_dx_12m, emp_dx_12m, ami_dx_12m, chf_dx_12m, dep_dx_12m, chl_dx_12, kid_dx_12m|
|Q36 take medication for some disease| dia_rx_12m, ast_rx_12m, hbp_rx_12m, emp_rx_12m, ami_rx_12m, chf_rx_12m, dep_rx_12m, chil_rx_12m, kid_rx_12m|
|ever checked something? | Q37 chl_chk_12m, Q38 dia_chk_12m, Q39 mam_chk_12m, Q40 pap_chk_12m |
| Q41 smk_ever_12m | smoked at least 100 cigs in your entire life? |
|Q42 smk_curr_12m| do you currently smoke cigs?|
|Q43 smk_avg_mod_12m| average number of cigs per day |
|Q44 smk_quit_12m| advised to quit smoking?|
|Q45 female_12m| - |
|Q46 birthyear_12m | - |
|Q47 employ_det_12m| employed or self_employed. It derives *employ_12m*|
|Q48 employ_hrs_12m | average hours worked per week|
|Q49 hhinc_cat_12m| household income category| 
|hhinc_pctfpl_12m | household income as percent of federal poverty line. This value is derived. |
|Q50-Q51 race| race_hisp_12m, race_white_12m, race_amerindian_12m, race_asian_12m, race_pacific_12m, race_other_qn_12m|
|Q52 edu_12m| highest level of education|
|Q53 live arragement| live_partner_12m, live_alone_12m, live_parents_12m, live_relatives_12m, live_friends_12m, live_other_12m|
|Q54 hhsize_12m| household size (adults and children)|
|Q55 num19_12m| number of family members under 19 in the house|
||corrections|
|needmet_med_cor_12m |derived. Got all needed medical care in last 6 months or no need. |
|needmet_rx_cor_12m | derived. Got all needed prescriptions in last 6 months, or no rx care needed|
|needmet_dent_cor_12m | Derived. Got all needed dental care in last 6 month or no dental care needed.|








In [3]:
s12.head().T

,0,1,2,3,4
person_id,1,2,3,4,5
sample_12m,In 12m mail survey sample,In 12m mail survey sample,In 12m mail survey sample,NOT in 12m mail survey sample,In 12m mail survey sample
dt_mail_12m,2009-08-11,2009-08-11,2009-08-03,NaN,2009-08-14
dt_returned_12m,2009-11-24,2009-08-31,NaN,NaN,2009-08-21
ret_mode_12m,Mail,Mail,NaN,NaN,Mail
...,...,...,...,...,...
hhsize_12m,3.0,2.0,NaN,NaN,3.0
num19_12m,0.0,1.0,NaN,NaN,NaN
needmet_med_cor_12m,No,No,NaN,NaN,Yes
needmet_rx_cor_12m,No,No,NaN,NaN,Yes


## Sanity check:
- Is the key able to link data? See if the identifiers line up across the data sources.
- How many variables?
- How many observations?

In [4]:
print(all(s12.person_id == descr.person_id))
print(all(s12.person_id == prgm.person_id))
# Is the function all() self-explained?

True
True


## Data Wrangling

This sub-section follows the textbook stuff.

Let's collect some key variables into a new data frame, `P`.

In [5]:
P = descr[["person_id", "household_id", "numhh_list"]].copy()
P["medicaid"] = pd.to_numeric(prgm["ohp_all_ever_firstn_30sep2009"] == "Enrolled")
P["selected"] = pd.to_numeric(descr["treatment"] == "Selected")
P["numhh_list"] = P["numhh_list"].replace({"signed self up": "1",
                                           "signed self up + 1 additional person": "2",
                                           "signed self up + 2 additional people": "3+"})

In [10]:
P.head()

,person_id,household_id,numhh_list,medicaid,selected
0,1,100001,1,False,True
1,2,100002,1,True,True
2,3,100003,1,False,False
3,4,100004,1,True,False
4,5,100005,1,False,True


In [6]:
P.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74922 entries, 0 to 74921
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   person_id     74922 non-null  int64 
 1   household_id  74922 non-null  int64 
 2   numhh_list    74922 non-null  object
 3   medicaid      74922 non-null  bool  
 4   selected      74922 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 1.9+ MB


The 12-month follow-up is the survey that really matters for outcomes that we are collecting into `Y`.

In [7]:
Y = s12[["weight_12m", "doc_any_12m", "doc_num_mod_12m", "er_any_12m", "er_num_mod_12m", "hosp_any_12m", "hosp_num_mod_12m"]].copy()

# Converting values in 'doc_any_12m', 'er_any_12m', and 'hosp_any_12m' columns to numeric
Y["doc_any_12m"] = pd.to_numeric(Y["doc_any_12m"] == "Yes")
Y["er_any_12m"] = pd.to_numeric(Y["er_any_12m"] == "Yes")
Y["hosp_any_12m"] = pd.to_numeric(Y["hosp_any_12m"] == "Yes")

In [8]:
Y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74922 entries, 0 to 74921
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   weight_12m        58405 non-null  float64
 1   doc_any_12m       74922 non-null  bool   
 2   doc_num_mod_12m   23477 non-null  float64
 3   er_any_12m        74922 non-null  bool   
 4   er_num_mod_12m    23517 non-null  float64
 5   hosp_any_12m      74922 non-null  bool   
 6   hosp_num_mod_12m  23575 non-null  float64
dtypes: bool(3), float64(4)
memory usage: 2.5 MB


In [10]:
Y.head()

,weight_12m,doc_any_12m,doc_num_mod_12m,er_any_12m,er_num_mod_12m,hosp_any_12m,hosp_num_mod_12m
0,1.0,False,0.0,False,0.0,False,0.0
1,1.0,False,0.0,False,0.0,False,0.0
2,0.0,False,NaN,False,NaN,False,NaN
3,NaN,False,NaN,False,NaN,False,NaN
4,1.0,False,0.0,False,0.0,False,0.0


There are a few covariates that will explain in heterogeneity in responses (or more importantly, treatment effects),or we care about them purely for descriptive purposes. We collect these into `X`.

In [13]:
X = s12.iloc[:, 120:147].copy()
X["dt_returned_12m"] = pd.to_datetime(s12["dt_returned_12m"])
X["dt_returned"] = X["dt_returned_12m"].dt.strftime("%Y-%m")

In [11]:
s12.columns[120:147]

Index(['smk_ever_12m', 'smk_curr_12m', 'smk_avg_mod_12m', 'smk_quit_12m',
       'female_12m', 'birthyear_12m', 'employ_12m', 'employ_det_12m',
       'employ_hrs_12m', 'hhinc_cat_12m', 'hhinc_pctfpl_12m', 'race_hisp_12m',
       'race_white_12m', 'race_black_12m', 'race_amerindian_12m',
       'race_asian_12m', 'race_pacific_12m', 'race_other_qn_12m', 'edu_12m',
       'live_partner_12m', 'live_alone_12m', 'live_parents_12m',
       'live_relatives_12m', 'live_friends_12m', 'live_other_12m',
       'hhsize_12m', 'num19_12m'],
      dtype='object')

In [14]:
X["dt_returned"].value_counts()

dt_returned
2009-08    10574
2009-09     4688
2009-07     3965
2009-10     2061
2009-11      708
2009-12      516
2010-02      483
2010-01      365
2010-03      262
2009-06      155
Name: count, dtype: int64

In [15]:
insurv = s12.index[s12["sample_12m_resp"] == "12m mail survey responder"]

# Select rows from X, Y, and P based on the insurv indices
X = X.loc[insurv].copy()
Y = Y.loc[insurv].copy()
P = P.loc[insurv].copy()

In [16]:
nan_counts = Y.apply(lambda y: y.isna().sum())

print(nan_counts)

weight_12m            0
doc_any_12m           0
doc_num_mod_12m     300
er_any_12m            0
er_num_mod_12m      260
hosp_any_12m          0
hosp_num_mod_12m    202
dtype: int64


In [17]:
# Finding indices of rows where there are no missing values in any column of Y
nomiss_indices = Y[~Y.apply(lambda y: y.isna().any(), axis=1)].index

# Selecting rows from X, Y, and P based on the nomiss_indices
X = X.loc[nomiss_indices].copy()
Y = Y.loc[nomiss_indices].copy()
P = P.loc[nomiss_indices].copy()

In [18]:
weights = Y.iloc[:,0].copy()
weights

0        1.0
1        1.0
4        1.0
5        1.0
7        1.0
        ... 
74904    1.0
74909    1.0
74910    1.0
74916    1.0
74920    1.0
Name: weight_12m, Length: 23107, dtype: float64

In [19]:
Y.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23107 entries, 0 to 74920
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   weight_12m        23107 non-null  float64
 1   doc_any_12m       23107 non-null  bool   
 2   doc_num_mod_12m   23107 non-null  float64
 3   er_any_12m        23107 non-null  bool   
 4   er_num_mod_12m    23107 non-null  float64
 5   hosp_any_12m      23107 non-null  bool   
 6   hosp_num_mod_12m  23107 non-null  float64
dtypes: bool(3), float64(4)
memory usage: 970.3 KB


In [20]:
Y = Y.iloc[:, :-1] # This is actually a dangerous snippet. Why?
Y

,weight_12m,doc_any_12m,doc_num_mod_12m,er_any_12m,er_num_mod_12m,hosp_any_12m
0,1.0,False,0.0,False,0.0,False
1,1.0,False,0.0,False,0.0,False
4,1.0,False,0.0,False,0.0,False
5,1.0,True,1.0,False,0.0,False
7,1.0,False,0.0,False,0.0,False
...,...,...,...,...,...,...
74904,1.0,True,4.0,False,0.0,False
74909,1.0,True,2.0,False,0.0,False
74910,1.0,True,2.0,False,0.0,False
74916,1.0,True,5.0,True,3.0,True


Replace some ridiculous values in survey and drop a variable

In [21]:
X['hhsize_12m'].value_counts()

hhsize_12m
2.0         6331
1.0         4223
3.0         4065
4.0         3477
5.0         2027
6.0          973
7.0          445
8.0          198
9.0           75
10.0          49
11.0          27
12.0          12
20.0           6
14.0           5
13.0           3
18.0           3
25.0           2
21.0           1
35.0           1
50.0           1
23.0           1
15.0           1
16.0           1
1000.0         1
163559.0       1
38.0           1
29.0           1
500.0          1
Name: count, dtype: int64

In [22]:
X.loc[X['hhsize_12m'] > 10, 'hhsize_12m'] = 10 # this needs discretion, be aware
# How does Matt how the magic number 10? He must have checked the distribution!
X.drop(columns=['num19_12m'], inplace=True) #remove a variable from dataFrame

Attach `doc_any` to `P` and organize things to make it pretty for text

In [23]:
P.columns

Index(['person_id', 'household_id', 'numhh_list', 'medicaid', 'selected'], dtype='object')

In [24]:
P['doc_any_12m'] = Y['doc_any_12m']
P = P.iloc[:, [0,1,5,4,3,2]]
P.rename(columns={P.columns[5]: 'numhh'}, inplace=True)

In [25]:
P.columns

Index(['person_id', 'household_id', 'doc_any_12m', 'selected', 'medicaid',
       'numhh'],
      dtype='object')

In [25]:
# intermediate data, not necessary
P.to_csv("P.csv", index=False)
X.to_csv("X.csv", index=False)
Y.to_csv("Y.csv", index=False)

# This is where the book starts analysis

Why?
- Most work is data processing.
- Models are usually well prepared for you.
- Make the data fit the function/package/library requirements.

A simple count of two groups.

In [26]:
P = pd.read_csv("https://www.dropbox.com/scl/fi/yyse8x40k9xyxti2qassp/P.csv?rlkey=yx32o46r3j9pnqoyve3936p1y&dl=1")
P.head()

,person_id,household_id,doc_any_12m,selected,medicaid,numhh
0,1,100001,False,True,False,1
1,2,100002,False,True,True,1
2,5,100005,False,True,False,1
3,6,100006,True,True,False,1
4,8,102094,False,False,False,2


In [27]:
P.selected.value_counts()

selected
False    11629
True     11478
Name: count, dtype: int64

In [28]:
Y = pd.read_csv("https://www.dropbox.com/scl/fi/peldi8mjzknsh0ggo43sh/Y.csv?rlkey=9yulu7fo5zszp3d5skjed3nqk&dl=1")

## ATE

In [30]:
ybar = P.groupby('selected')['doc_any_12m'].mean()
print(ybar)

# Calculating the Average Treatment Effect (ATE)
ATE = ybar.iloc[1] - ybar.iloc[0]
print(ATE)

selected
False    0.573480
True     0.630946
Name: doc_any_12m, dtype: float64
0.05746606499595419


Let's quantify the uncertainty in the estimate:

Can you recall the formula for standard error of the difference of sample means of independent samples?

In [31]:
nsel = P['selected'].value_counts()
yvar = P.groupby('selected')['doc_any_12m'].var()
seATE = np.sqrt(np.sum(yvar / nsel))
conf_interval = ATE + np.array([-2, 2]) * seATE

# Printing the results
print("Counts for 'selected':")
print(nsel)
print("Variance of 'doc_any_12m' by 'selected':")
print(yvar)
print("Standard Error of ATE:", seATE)
print("95% Confidence Interval for ATE:", conf_interval)

Counts for 'selected':
selected
False    11629
True     11478
Name: count, dtype: int64
Variance of 'doc_any_12m' by 'selected':
selected
False    0.244622
True     0.232873
Name: doc_any_12m, dtype: float64
Standard Error of ATE: 0.006428387291688661
95% Confidence Interval for ATE: [0.04460929 0.07032284]


## reweight

- why re-weighting?

In [32]:
fig = px.histogram(weights, nbins=20)
fig.update_layout(title='Histogram of Weights', xaxis_title='Weights', yaxis_title='Frequency')
fig.show()

In [33]:
P['weights'] = weights
nsel_w = P.groupby('selected')['weights'].sum()
print(nsel_w)


selected
False    4428.970992
True     4332.082531
Name: weights, dtype: float64


In [40]:
ybar_w = P.groupby('selected')[['weights', 'doc_any_12m']].apply(
    lambda group: np.sum(group['weights'] * group['doc_any_12m']) / nsel_w[group.name],
    )
print(ybar_w)

ATEweighted = ybar_w.iloc[1] - ybar_w.iloc[0]
print(ATEweighted)

selected
False    0.569808
True     0.636503
dtype: float64
0.06669442494394962


## Explore more outcomes

This is a typical empirical strategy. For example, if you want to see if a new economic policy works well, there could be MANY indicators to represent the economy. A simple solution is to **try them all**.

In [41]:
ybar = Y.apply(lambda col: col.groupby(P['selected']).mean())
ybar

,weight_12m,doc_any_12m,doc_num_mod_12m,er_any_12m,er_num_mod_12m,hosp_any_12m
selected,,,,,,
False,1.220840,0.573480,1.805142,0.251612,0.437441,0.068020
True,1.238318,0.630946,2.055672,0.249085,0.428995,0.068566


In [42]:
yvar = Y.apply(lambda col: col.groupby(P['selected']).var())
yvar

,weight_12m,doc_any_12m,doc_num_mod_12m,er_any_12m,er_num_mod_12m,hosp_any_12m
selected,,,,,,
False,0.365953,0.244622,7.732581,0.188320,0.934962,0.063398
True,0.378454,0.232873,8.792840,0.187058,0.893058,0.063870


In [43]:
ATEs = ybar.loc[True, :] - ybar.loc[False, :]
ATEs

weight_12m         0.017477
doc_any_12m        0.057466
doc_num_mod_12m    0.250529
er_any_12m        -0.002527
er_num_mod_12m    -0.008446
hosp_any_12m       0.000546
dtype: float64

In [44]:
nsel = P['selected'].value_counts()
nsel

selected
False    11629
True     11478
Name: count, dtype: int64

In [45]:
ATEs_se = np.sqrt(yvar.loc[True, :] / nsel[True] + yvar.loc[False, :] / nsel[False])
ATEs_se

weight_12m         0.008028
doc_any_12m        0.006428
doc_num_mod_12m    0.037829
er_any_12m         0.005700
er_num_mod_12m     0.012578
hosp_any_12m       0.003319
dtype: float64

In [46]:
# estimate, se and t stats
df0 = pd.DataFrame({"AET": ATEs, "se": ATEs_se, "t-stat": ATEs/ATEs_se})

In [47]:
df0

,AET,se,t-stat
weight_12m,0.017477,0.008028,2.177182
doc_any_12m,0.057466,0.006428,8.939422
doc_num_mod_12m,0.250529,0.037829,6.622759
er_any_12m,-0.002527,0.005700,-0.443351
er_num_mod_12m,-0.008446,0.012578,-0.671514
hosp_any_12m,0.000546,0.003319,0.164607


## Households

In [48]:
P_hh = P.drop_duplicates('household_id')
P_hh

,person_id,household_id,doc_any_12m,selected,medicaid,numhh,weights
0,1,100001,False,True,False,1,1.0
1,2,100002,False,True,True,1,1.0
2,5,100005,False,True,False,1,NaN
3,6,100006,True,True,False,1,NaN
4,8,102094,False,False,False,2,1.0
...,...,...,...,...,...,...,...
23100,74900,174900,True,True,False,1,NaN
23102,74905,174905,True,False,False,1,NaN
23103,74910,174910,True,False,False,1,NaN
23105,74917,174917,True,False,False,1,NaN


In [49]:
Y_hh = Y.loc[P_hh.index,:]
Y_hh

,weight_12m,doc_any_12m,doc_num_mod_12m,er_any_12m,er_num_mod_12m,hosp_any_12m
0,1.0,False,0.0,False,0.0,False
1,1.0,False,0.0,False,0.0,False
2,1.0,False,0.0,False,0.0,False
3,1.0,True,1.0,False,0.0,False
4,1.0,False,0.0,False,0.0,False
...,...,...,...,...,...,...
23100,1.0,True,3.0,True,1.0,False
23102,1.0,True,4.0,False,0.0,False
23103,1.0,True,2.0,False,0.0,False
23105,1.0,True,5.0,True,3.0,True


In [50]:
ybar = Y_hh.apply(lambda col: col.groupby(P_hh['selected']).mean())
ybar

,weight_12m,doc_any_12m,doc_num_mod_12m,er_any_12m,er_num_mod_12m,hosp_any_12m
selected,,,,,,
False,1.234531,0.579173,1.844413,0.259058,0.453352,0.070376
True,1.260679,0.641502,2.122076,0.255798,0.442727,0.070876


In [51]:
yvar = Y_hh.apply(lambda col: col.groupby(P_hh['selected']).var())
yvar

,weight_12m,doc_any_12m,doc_num_mod_12m,er_any_12m,er_num_mod_12m,hosp_any_12m
selected,,,,,,
False,0.384070,0.243755,8.010791,0.191965,0.970313,0.065429
True,0.408136,0.230000,9.135096,0.190384,0.917829,0.065860


In [52]:
AETs = ybar.loc[True, :] - ybar.loc[False, :]
AETs

weight_12m         0.026148
doc_any_12m        0.062329
doc_num_mod_12m    0.277663
er_any_12m        -0.003261
er_num_mod_12m    -0.010626
hosp_any_12m       0.000501
dtype: float64

In [53]:
nsel = P_hh['selected'].value_counts()
nsel

selected
False    10515
True      9961
Name: count, dtype: int64

In [54]:
AETs_se = np.sqrt(yvar.loc[True, :] / nsel[True] + yvar.loc[False, :] / nsel[False])
AETs_se

weight_12m         0.008803
doc_any_12m        0.006802
doc_num_mod_12m    0.040975
er_any_12m         0.006113
er_num_mod_12m     0.013580
hosp_any_12m       0.003582
dtype: float64

In [55]:
# estimate, se and t stats
df1 = pd.DataFrame({"AET": AETs, "se": AETs_se, "t-stat": AETs/AETs_se})

In [56]:
df1

,AET,se,t-stat
weight_12m,0.026148,0.008803,2.970255
doc_any_12m,0.062329,0.006802,9.162924
doc_num_mod_12m,0.277663,0.040975,6.776450
er_any_12m,-0.003261,0.006113,-0.533429
er_num_mod_12m,-0.010626,0.013580,-0.782443
hosp_any_12m,0.000501,0.003582,0.139781


## Balance Check
Maybe we worry about the imbalance in `numhh`:

- What is `numhh`? Recall...
- Any pattern from the following table?
- Any explanation for such a pattern?


In [57]:
selected_counts = P['selected'].value_counts()
selected_counts

selected
False    11629
True     11478
Name: count, dtype: int64

In [58]:
contingency_table = pd.crosstab(P['selected'], P['numhh'])
contingency_table

numhh,1,2,3+
selected,,,
False,8684,2939,6
True,7525,3902,51


In [59]:
chi2, p_value, dof, expected = chi2_contingency(contingency_table)

# Printing the results
print("Chi-squared statistic:", chi2)
print("P-value:", p_value)
print("Degrees of freedom:", dof)
print("Expected frequencies table:")
print(expected)

Chi-squared statistic: 252.98334621246406
P-value: 1.1624234518501736e-55
Degrees of freedom: 2
Expected frequencies table:
[[8157.46141862 3442.85233912   28.68624226]
 [8051.53858138 3398.14766088   28.31375774]]


## Regression for Control

If something is bothering your inference, the easiest way to get around is modelling it. Of course, this is not without error or trouble. But it is usually convincing and straightforward to communicate. So let's build regression models of any outpatient visits.

In [60]:
P.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23107 entries, 0 to 23106
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   person_id     23107 non-null  int64  
 1   household_id  23107 non-null  int64  
 2   doc_any_12m   23107 non-null  bool   
 3   selected      23107 non-null  bool   
 4   medicaid      23107 non-null  bool   
 5   numhh         23107 non-null  object 
 6   weights       7161 non-null   float64
dtypes: bool(3), float64(1), int64(2), object(1)
memory usage: 789.9+ KB


In [61]:
P['doc_any_12m'] = P['doc_any_12m'].astype(int)
model = smf.ols(formula='doc_any_12m ~ selected + numhh', data=P)
# Fitting the model
result = model.fit()

# Printing the summary of the coefficients in a tabular format
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:            doc_any_12m   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     57.43
Date:                Tue, 19 Aug 2025   Prob (F-statistic):           5.62e-37
Time:                        21:20:16   Log-Likelihood:                -16194.
No. Observations:               23107   AIC:                         3.240e+04
Df Residuals:                   23103   BIC:                         3.243e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.5902      0.005  

Below is a demeaning-and-interacting way to get the ATE after a covariate adjustment; see equation (5.7)

$$E(y\mid d, numhh) = \alpha_{numhh} + d\gamma_{numhh}$$

- This is equivalent to using subsamples to fit the regression separately.
- Using interactions captures the heterogeneity better.
- What is ATE in terms of the model parameters?

In [62]:
P['doc_any_12m'] = P['doc_any_12m'].astype(int)
model = smf.ols(formula='doc_any_12m ~ selected * numhh', data=P)
# Fitting the model
result = model.fit()

# Printing the summary of the coefficients in a tabular format
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:            doc_any_12m   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     35.46
Date:                Tue, 19 Aug 2025   Prob (F-statistic):           2.79e-36
Time:                        21:20:47   Log-Likelihood:                -16191.
No. Observations:               23107   AIC:                         3.239e+04
Df Residuals:                   23101   BIC:                         3.244e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept       

Look at households one at a time.

In [63]:
P.numhh.unique()

array(['1', '2', '3+'], dtype=object)

In [64]:
P0 = P[P['numhh']=='1']
model = smf.ols(formula='doc_any_12m ~ selected', data=P0)
# Fitting the model
result = model.fit()

# Printing the summary of the coefficients in a tabular format
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:            doc_any_12m   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     86.98
Date:                Tue, 19 Aug 2025   Prob (F-statistic):           1.24e-20
Time:                        21:21:07   Log-Likelihood:                -11241.
No. Observations:               16209   AIC:                         2.249e+04
Df Residuals:                   16207   BIC:                         2.250e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.5868      0.005  

# Exercise

Estimate ATE at the household level.
- data `P_hh` is in the *Households* subsection.
- Use `doc_any_12m` as the dependent variable.
- Use `selected`, `numhh` and their interactions as the independent variables.
- (this is we care the most!) Explain these variable's meanings and the estimation result.